# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [41]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [42]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [43]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [44]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [45]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [46]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity 
                    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [47]:
try:
    session.set_keyspace('udacity')

except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1: modeled with the minimum number of columns for fast writes and in column order to optimize the fetch statement

In [48]:
## Query_1 provides the minimal amount of columns neccessary to be able to analyze the artist, song title and song length 
## For a given sessionId en itemInSession. The primary key needs a combination of sessionId and itemInSession because either
## alone isn't unique and songs can be listened to multiple times per user and session.

query_1 = "SELECT artist, song, length FROM event_library_1_session_details WHERE sessionId = 338 AND itemInSession = 4"

query_table_1 = "CREATE TABLE IF NOT EXISTS event_library_1_session_details (sessionId int,\
                                                                             itemInSession int,\
                                                                             artist text,\
                                                                             song text,\
                                                                             length float,\
                                                                             PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute("DROP TABLE IF EXISTS event_library_1_session_details")
except Exception as e:
    print(e)
try:
    session.execute(query_table_1)
except Exception as e:
    print(e)

In [49]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_library_1_session_details (sessionId,\
                                                              itemInSession,\
                                                              artist,\
                                                              song,\
                                                              length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0],  line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [50]:
# To test whether each item is inserted, the SELECT statement contains the same columns as the insert statement.
# The query is limited to 1 row as this is for verification purposes only
try:
    rows = session.execute("SELECT sessionId, itemInSession, artist, song, length FROM event_library_1_session_details LIMIT 1")
except Exception as e:
    print(e)

for row in rows:
    print(row.sessionid,
          row.iteminsession,
          row.artist,
          row.song,
          row.length)
    
try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)
print()

for row in rows:
    print(row.artist,\
          row.song,\
          row.length)

23 0 Regina Spektor The Calculation (Album Version) 191.08526611328125

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

#### Query 2: modeled with the minimum number of columns for fast writes and in column order to optimize the fetch statement

In [51]:
## Query_2 provides the minimal amount of columns neccessary to be able to analyze the artist name, song sorted by itemInSession,
## and first and last name for a given userId and sessionId. The primary key needs a combination of userId and sessionId because either
## alone isn't unique and sufficient to distinguish between multiple songs per session per user. ItemInSession is part of the key
## in order to fulfill this requirement.

query_2 = "SELECT artist, song, length, firstname, lastname FROM event_library_2_user_details WHERE userid = 10 AND sessionid = 182"

query_table_2 = "CREATE TABLE IF NOT EXISTS event_library_2_user_details (userId int,\
                                                                          sessionId int,\
                                                                          itemInSession int,\
                                                                          artist text,\
                                                                          song text,\
                                                                          length float,\
                                                                          firstName text,\
                                                                          lastName text,\
                                                                          PRIMARY KEY((userId, sessionId), itemInSession))\
                                                                          WITH CLUSTERING ORDER BY (itemInSession DESC)"

try:
    session.execute("DROP TABLE IF EXISTS event_library_2_user_details")
except Exception as e:
    print(e)
try:
    session.execute(query_table_2)
except Exception as e:
    print(e)

In [52]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_library_2_user_details (userId,\
                                                           sessionId,\
                                                           itemInSession,\
                                                           artist,\
                                                           song,\
                                                           length,\
                                                           firstName,\
                                                           lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], float(line[5]), line[1], line[4]))

In [31]:
# To test whether each item is inserted, the SELECT statement contains the same columns as the insert statement.
# The query is limited to 1 row as this is for verification purposes only
try:
    rows = session.execute("SELECT userId,\
                                   sessionId,\
                                   itemInSession,\
                                   artist,\
                                   song,\
                                   length,\
                                   firstName,\
                                   lastName FROM event_library_2_user_details LIMIT 1")
except Exception as e:
    print(e)

for row in rows:
    print(row.userid,
          row.sessionid,
          row.iteminsession,
          row.artist,
          row.song,
          row.length,
          row.firstname,
          row.lastname)
print()

## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist,
          row.song,
          row.firstname,
          row.lastname)

58 768 2 Evergreen Terrace Zero 172.381591796875 Emily Benson

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz


#### Query 3: modeled with the minimum number of columns for fast writes and in column order to optimize the fetch statement

In [32]:
## Query_3 provides the minimal amount of columns neccessary to be able to analyze the users' first and last name for a given song.
## Since people can have the same full name and listen to the same song, a combination of song and userId are chosen as the primary Key.
query_3 = "SELECT firstname, lastname FROM event_library_3_users_per_song WHERE song = 'All Hands Against His Own'"

query_table_3 = "CREATE TABLE IF NOT EXISTS event_library_3_users_per_song (song text,\
                                                                            firstName text,\
                                                                            lastName text,\
                                                                            artist text,\
                                                                            userId int,\
                                                                            PRIMARY KEY((song), userId))"

try:
    session.execute("DROP TABLE IF EXISTS event_library_3_users_per_song")
except Exception as e:
    print(e)
try:
    session.execute(query_table_3)
except Exception as e:
    print(e)

In [34]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_library_3_users_per_song (song,\
                                                             firstName,\
                                                             lastName,\
                                                             artist,\
                                                             userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], line[0], int(line[10])))

In [35]:
# To test whether each item is inserted, the SELECT statement contains the same columns as the insert statement.
# The query is limited to 1 row as this is for verification purposes only
try:
    rows = session.execute("SELECT song, firstName, lastName, artist FROM event_library_3_users_per_song LIMIT 1")
except Exception as e:
    print(e)

for row in rows:
        print(row.song,
              row.firstname,
              row.lastname,
              row.artist)
print()

try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname,
          row.lastname)

Wonder What's Next Chloe Cuevas Chevelle

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [39]:
for table in ['event_library_1_session_details',
              'event_library_2_user_details',
              'event_library_3_users_per_song']:
    try:
        rows = session.execute(f"DROP TABLE IF EXISTS {table}")
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [40]:
session.shutdown()
cluster.shutdown()